# Workgroup 6 - Group 7

In [8]:
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy
import keras

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors


from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

In this case, we test the convergence hypothesis proposed by Solow-Swan. We propose the following model:

  $$
  Y = \beta_1 D +  \beta_2'X + \epsilon.
  $$
  
Where $Y$ is the growth rate of a certain country, $D$ is the initial wealth level of said country, and $X$ is a set of control variables related to each country's institutional, educational, and similar characteristics.

In [9]:
# Importing Data
rdata = pyreadr.read_r("data/GrowthData.RData")
data = rdata['GrowthData']
data

,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,1,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,1,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,1,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,1,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195
4,0.027930,1,7.162397,0.1740,0.299252,0.037367,0.017,0.027,0.007,0.82,...,2.11,0.257,0.287,0.229,34.5,17.6,0.1211,0.1308,2.500,0.003283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.031196,1,8.991064,0.0000,0.371898,0.014586,0.255,0.336,0.170,0.98,...,11.41,2.226,2.494,1.971,27.5,15.9,0.4407,0.4257,2.529,-0.011883
86,0.034096,1,8.025189,0.0050,0.296437,0.013615,0.108,0.117,0.093,1.00,...,1.95,0.510,0.694,0.362,20.2,15.7,0.1669,0.2201,25.553,-0.039080
87,0.046900,1,9.030137,0.0000,0.265778,0.008629,0.288,0.337,0.237,1.00,...,25.64,2.727,2.664,2.788,20.4,9.4,0.3238,0.3134,4.152,0.005175
88,0.039773,1,8.865312,0.0000,0.282939,0.005048,0.188,0.236,0.139,1.00,...,10.76,1.888,1.920,1.860,20.0,16.0,0.1845,0.1940,0.452,-0.029551


In [10]:
#Defining Variables
Y = data['Outcome']
X = data.drop('Outcome', 1)
D = data[['intercept', 'gdpsh465']]

<ipython-input-10-44840a64cdaa>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = data.drop('Outcome', 1)


## OLS without covariates
***
We estimate a model of the outcome variable just on the treatment variable (initial wealth). That is:

 $$
  Y = \beta_1 D  + \epsilon.
  $$

In [11]:
# OLS w/o controls
model = "Outcome ~ gdpsh465"
baseline_ols = smf.ols(model , data=data).fit()
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

[0.025   -0.010810
0.975]    0.013444
Name: gdpsh465, dtype: float64


Coef.       0.001317
Std.Err.    0.006102
t           0.215777
P>|t|       0.829661
[0.025     -0.010810
0.975]      0.013444
Name: gdpsh465, dtype: float64

## OLS with Covariates
Now we estimate de model as initially proposed:

In [12]:
# OLS with controls
y = 'Outcome'

data_columns = list(data)
no_relev_col = ['Outcome',  'gdpsh465']

# This gives us: new_list = ['carrot' , 'lemon']
z = [col for col in data_columns if col not in no_relev_col]

control_formula = "Outcome" + ' ~ ' + 'gdpsh465 + ' + ' + '.join( z )


control_ols = smf.ols( control_formula , data=data).fit()
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

[0.025   -0.070600
0.975]    0.051844
Name: gdpsh465, dtype: float64


Coef.      -0.009378
Std.Err.    0.029888
t          -0.313774
P>|t|       0.756019
[0.025     -0.070600
0.975]      0.051844
Name: gdpsh465, dtype: float64

In [13]:
raw_res    = baseline_ols_table.loc[['gdpsh465'], :]
control_res = control_ols_table.loc[['gdpsh465'], :]

new_indexes = ['OLS - W/O controls', 'OLS - With controls']

table_ols = raw_res.append(control_res)

table_ols.index = new_indexes
table_ols

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
OLS - W/O controls,0.001317,0.006102,0.215777,0.829661,-0.01081,0.013444
OLS - With controls,-0.009378,0.029888,-0.313774,0.756019,-0.07060,0.051844


We notice that both OLS estimations give no significant results. That is because we have $90$ countries, but $63$ control variables, which means we have a relatively large number of variables for a relatively small number of observations ($\frac{p}{n}$ not small), therefore the OLS method performs poorly, with or without covariates.

In [15]:
data.shape

(90, 63)

## DML using Lasso
***
First we define the DML function

In [12]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
    ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil ), axis = 1), columns = ['ytil','dtil'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result
    

In [13]:
Y = data['Outcome']
D = data['gdpsh465']
X = data.drop(['Outcome', 'gdpsh465'], axis=1)
#CLU = data['CountyCode']

In [14]:
# as matrix
y = Y.to_numpy().reshape( len(Y) , 1 )
d = D.to_numpy().reshape( len(Y) , 1 )
x = X.to_numpy()
#clu = clu.to_numpy().reshape( len(Y) , 1 )

In [15]:
def dreg(x,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, d)
    return result

def yreg(x,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, y)
    return result

DML2_lasso = DML2_for_PLM(x, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.015364284846020522, SE is equal to 0.013084632901594081


C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8819039484553006, tolerance: 0.0066002110963832555
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.011572455203789687, tolerance: 1.9481138701417832e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7086931138131208, tolerance: 0.006359872444937687
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceW

## DML using Post-Lasso
***
Now we will perform the DML algorithm in a post-lasso context


In [25]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

In [26]:
def dreg(x,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , d )
    return result

def yreg( x , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , y )
    return result

DML2_lasso_post = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7052034218714942, tolerance: 0.0064954511306218495
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7352467620324676, tolerance: 0.0064954511306218495
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01340878276278119, tolerance: 2.229516308165847e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\envs\renv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWa

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.022908952699243732, SE is equal to 0.011757157493360372


## DML with Elastic Net 
***

In [27]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [28]:
# DML with cross-validated Elastic Net:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, y )
    return result

DML2_elnet = DML2_for_PLM(x, d, y, dreg, yreg, 2 )

C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  
Coefficient is -0.04473856775230579, SE is equal to 0.012256020660717949


## DML ridge
***

In [29]:
#DML with cross-validated Ridge:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_ridge = DML2_for_PLM(x, d, y, dreg, yreg, 2)

C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  
Coefficient is -0.004108022155506074, SE is equal to 0.009973034782523245


## DML Random forest
***

Now we are going to predict $y$ and $d$ using Random forest. Note that we have to set the maximun number of features. The number of features is going to be 61 since we have 63 variables and we cannot include $y$ or $d$ as features.

In [30]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [31]:
#DML with Random Forest:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(x, d, y, dreg, yreg, 2)   # set to 2 due to computation time

<ipython-input-31-10f97f9b3f09>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-31-10f97f9b3f09>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


0  


<ipython-input-31-10f97f9b3f09>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-31-10f97f9b3f09>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 61 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


1  
Coefficient is -0.032367707723629464, SE is equal to 0.012010075390869464


In [32]:
mods = [DML2_lasso, DML2_lasso_post ,DML2_elnet, DML2_ridge, DML2_RF]
mods_name = ["DML2_lasso", "DML2_lasso_post" ,'DML2_elnet', 'DML2_ridge', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_lasso,DML2_lasso_post,DML2_elnet,DML2_ridge,DML2_RF
RMSEY,0.002021,0.000422,3.700743e-18,8.943463e-18,0.001525
RMSED,0.026099,0.034598,1.480297e-16,1.154632e-15,0.024790


It looks like the best method for predicting both Y and D is Elastic Net.

## Best Method

In [38]:
# DML with cross-validated Elastic Net:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, y )
    return result

DML2_best = DML2_for_PLM(x, d, y, dreg, yreg, 2 )

C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\envs\renv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  
Coefficient is -0.058295049580833196, SE is equal to 0.015587324159142883


In [39]:
table = np.zeros( ( 8 , 2 ))
table[ 0 , 0] = table_ols.iloc[ 0 , 0 ]
table[ 1 , 0] = table_ols.iloc[ 1 , 0 ]
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_ridge['coef_est']
table[ 5 , 0] = DML2_elnet['coef_est']
table[ 6 , 0] = DML2_RF['coef_est']
table[ 7 , 0] = DML2_best['coef_est']
table[ 0 , 1] = table_ols.iloc[ 0 , 1 ]
table[ 1 , 1] = table_ols.iloc[ 1 , 1 ]
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_ridge['se']
table[ 5 , 1] = DML2_elnet['se']
table[ 6 , 1] = DML2_RF['se']
table[ 7 , 1] = DML2_best['se']

In [40]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "Least Squares with controls", "Lasso", \
             "Post-Lasso","CV Ridge", "CV Elastic Net", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.001,0.006
Least Squares with controls,-0.009,0.030
Lasso,0.015,0.013
Post-Lasso,0.023,0.012
CV Ridge,-0.004,0.010
CV Elastic Net,0.001,0.013
Random Forest,-0.032,0.012
Best,-0.058,0.016
